In [258]:
import spacy


In [259]:
def relative_clause(sentence):
    nlp = spacy.load("en_core_web_sm")
    doc=nlp(sentence)
    rel=False
    list_rel=['relcl','advcl']
    sent=[token for token in doc]
    r_clause=[]
    place=0
    for token in doc:
        #print(token.i,token.text,token.pos_, token.tag_,token.head.i,token.head,token.dep_)
        if not rel:
            if token.tag_ == 'WDT' or token.tag_=='WP':
                #print('WDT')
                if doc[token.i-1].pos_=='PUNCT':
                    sent.remove(doc[token.i-1])
                h=token.head
                sub=[elt for elt in h.subtree]
                if (len(doc)-1)>sub[len(sub)-1].i and sub[0].i!=0:
                    if (doc[sub[0].i-1].text and doc[sub[len(sub)-1].i+1].text)== ',':
                        sent=[elt for elt in sent if (elt not in sub) and (elt !=doc[sub[len(sub)-1].i+1])]
                        continue
                     
                if h.tag_=='VBZ' and h.head.tag_ == 'VBZ':
                    place= h.head.i
                    idx=sent.index(h.head)
                    for elt in h.subtree:
                        sent.remove(elt)
                        if elt.i<h.i:
                            if elt !=token and elt.tag_!='PRP':
                                r_clause.append(elt)
                        else:
                            r_clause.append(elt)
                    r_clause.append(nlp('to')[0])
                    sent[idx]=nlp(h.head.lemma_)[0]
                    
                            
                elif h.lemma_== 'be' and h.head.tag_ in ('NN', 'NNS'):
                    if doc[h.head.i-1].tag_=='DT':
                        place= h.head.i
                    elif doc[h.head.i-1].tag_=='JJ':
                        place= h.head.i-1
                    for elt in h.subtree:
                        sent.remove(elt)
                        if elt!=token and elt!= h:
                            r_clause.append(elt)
                
                elif h.pos_=='VERB' and h.head.tag_ in ('NN', 'NNS'):
                    subj=[elt for elt in h.head.subtree if elt not in sub]
                    subj[0]=nlp(subj[0].text.capitalize())[0]
                    for i, elt in enumerate(sub):
                        sent.remove(elt)
                        if i!=0:
                            r_clause.append(elt)
                    r_clause= subj+r_clause
                    r_clause.append(nlp('.')[0])
                    if sent[len(sent)-1].pos_!='PUNCT':
                        sent.append(nlp('.')[0])
                    
                    place=len(sent)
                rel=True
            elif token.dep_ in list_rel:
                #print('ADVCL')
                h=token.head
                if doc[h.head.i].lemma_ == 'be':
                    
                    sent.insert(h.head.i-1,nlp('that')[0])
                    rel_rec=''
                    for i,elt in enumerate(token.subtree):
                        sent.remove(elt)
                        if i==0:
                            rel_rec+= elt.text.capitalize()+' '
                        else:
                            rel_rec+= elt.text+' '
                    for i,elt in enumerate(sent):
                        if i==0:
                            rel_rec+=elt.text.lower()+' '
                        else:
                            rel_rec+=elt.text+' '

                    r_clause,sent,place,t =relative_clause(rel_rec)

                    
                rel=True
   

    final_sent=[elt for elt in sent]
    for elt in r_clause[::-1]:
        final_sent.insert(place,elt)

    text=""
    for i,elt in enumerate(final_sent):
        text+=elt.text
        if i!= len(final_sent)-1 and final_sent[i+1].pos_!='PUNCT':
            text+=" "
            

                     
    return(r_clause,sent,place,text)          

a=relative_clause('I see a man that is tall and big') #ok

b=relative_clause('The treasure the pirates found was empty') # ok

c=relative_clause('I bought a new car that is very fast.') #ok

d=relative_clause('She lives in New York, which she hates.') #ok

e=relative_clause("I'd like to paint the room a color that is cheerful") #ok

f=relative_clause("The play, which debuted last week, has been sold out every night") #ok

g=relative_clause("I’m buying a gift for the neighbor who helped me move") #ok

print(a[3])
print(b[3])
print(c[3])
print(d[3])
print(e[3])
print(f[3])
print(g[3])

I see a tall and big man
The pirates found the empty treasure
I bought a very fast new car.
She hates to live in New York.
I 'd like to paint the room a cheerful color
The play has been sold out every night
I ’m buying a gift for the neighbor. The neighbor helped me move.
